# Auditoría de integridad (ejemplo)

Este notebook sirve como ejemplo para **leer auditorías** (salidas de `hdfs fsck`) y construir una **tabla resumen**.

✅ Tarea del alumnado: adaptar las rutas y completar los scripts para que las auditorías se guarden en un directorio accesible desde Jupyter.


## 1) Configuración de rutas

En el entorno del aula, el NameNode monta un volumen (por ejemplo `./notebooks` → `/media/notebooks`).
La idea es que el script `30_fsck_audit.sh` deje copias de auditoría en una ruta que Jupyter pueda leer.


In [3]:
from pathlib import Path # Librería para gestionar rutas de archivos de forma multiplataforma
import pandas as pd # Librería para manipulación de datos en tablas (DataFrames)
import re   # Librería para expresiones regulares (búsqueda de patrones de texto)

# Definimos la ruta de la carpeta que contiene los reportes de auditoría
# Path se encarga de que la ruta funcione tanto en Windows como en Linux/Docker
# Aqui hay que cambiar la fecha de la carpeta a la queremos referenciar, las carpetas se crean en esta misma pero se le pone la fecha con de la que 
# se a creado
AUDIT_DIR = Path("./2026-02-13")


# Comprobamos si la ruta existe físicamente en el sistema de archivos
# Esto devolverá True si la carpeta '2026-02-12' está en el directorio actual
print('AUDIT_DIR exists:', AUDIT_DIR.exists())
# Listamos y mostramos los primeros 5 archivos encontrados dentro de esa carpeta
# sorted() asegura que los archivos aparezcan en orden alfabético
# AUDIT_DIR.glob('*') busca todos los elementos dentro del directorio
print(sorted(AUDIT_DIR.glob('*'))[:5])


AUDIT_DIR exists: True
[WindowsPath('2026-02-13/iot_20260213.jsonl'), WindowsPath('2026-02-13/logs_20260213.log')]


## 2) Parseo básico de fsck

Contabilizamos palabras clave típicas:
- `CORRUPT`
- `MISSING`
- `Under replicated`


In [4]:
import re

def parse_fsck_text(text: str):
    """
    Analiza el contenido de un reporte FSCK y cuenta las incidencias.
    Usa expresiones regulares para encontrar palabras clave independientemente de mayúsculas/minúsculas.
    """
    return {
        # \b asegura que coincida la palabra exacta (límite de palabra), evitando falsos positivos
        'CORRUPT': len(re.findall(r'\bCORRUPT\b', text, flags=re.IGNORECASE)),
        # Busca bloques perdidos (MISSING)
        'MISSING': len(re.findall(r'\bMISSING\b', text, flags=re.IGNORECASE)),
        # Busca bloques con menos réplicas de las necesarias (Under replicated)
        'UNDER_REPLICATED': len(re.findall(r'Under replicated', text, flags=re.IGNORECASE)),
    }

# Lista para acumular los diccionarios de métricas de cada archivo
rows = []
# Iteramos sobre todos los archivos con extensión .log dentro de la carpeta de auditoría
# Nota: Asegúrate de que AUDIT_DIR esté definido previamente como un objeto Path
# Buscamos directamente archivos .log o .txt en la carpeta
for fsck_file in AUDIT_DIR.glob('*.log'): 
    # Leemos el contenido del archivo ignorando errores de caracteres especiales
    text = fsck_file.read_text(encoding='utf-8', errors='ignore')
    # Extraemos las métricas usando la función definida arriba
    metrics = parse_fsck_text(text)
    # Guardamos el nombre del archivo (o la fecha) para identificar el registro en la tabla
    metrics['dt'] = fsck_file.name  # Usamos el nombre del archivo como identificador
    # Añadimos los resultados a nuestra lista de filas
    rows.append(metrics)

# Si se encontraron datos, creamos el DataFrame y lo ordenamos por nombre/fecha
# Si la lista está vacía, creamos un DataFrame con las columnas vacías para evitar errores de visualización
df = pd.DataFrame(rows).sort_values('dt') if rows else pd.DataFrame(columns=['dt','CORRUPT','MISSING','UNDER_REPLICATED'])
# Mostramos el resultado final en formato tabla
df

,CORRUPT,MISSING,UNDER_REPLICATED,dt
0,0,0,0,logs_20260213.log


## 3) Exportación

Exporta la tabla a CSV para incluirla en el informe.


In [5]:
# Definimos la ruta y el nombre del archivo de salida usando Path para que sea compatible 
# tanto en Windows como en Linux/Docker automáticamente
out = Path('./audit_summary.csv')
# Exportamos el DataFrame (df) a un archivo CSV
# index=False: Evita que Pandas guarde la columna de índices numérica a la izquierda, 
# dejando un archivo más limpio solo con los datos de auditoría
df.to_csv(out, index=False)
# Al poner la variable 'out' al final, Jupyter imprimirá la ruta confirmando 
# donde se ha guardado físicamente el reporte
out


WindowsPath('audit_summary.csv')